In [10]:
import pandas as pd
df0 = pd.read_csv('master.txt')

def quickpeek(df):
  print(df.shape)
  print(df['player'].nunique(), 'QBs')
  display(df.head())

quickpeek(df0)

(10076, 19)
262 QBs


,player,date,team,home,opp,game,week,day,completions,passatt,passyards,passtds,ints,sacks,sackyards,rushatt,rushyards,rushtds,fumbles
0,Geno Smith,2013-12-01,NYJ,1,MIA,12,13,Sun,4,10,29,0,1,1.0,8.0,1,2,0,0
1,Ryan Tannehill,2013-12-01,MIA,0,NYJ,12,13,Sun,28,43,331,2,1,1.0,3.0,3,22,0,0
2,Brandon Weeden,2013-12-01,CLE,1,JAX,12,13,Sun,24,40,370,3,2,3.0,28.0,2,5,0,2
3,Joe Flacco,2013-11-28,BAL,1,PIT,12,13,Thu,24,35,251,1,0,2.0,14.0,4,7,0,1
4,Matt Flynn,2013-11-28,GNB,0,DET,12,13,Thu,10,20,139,0,1,7.0,37.0,2,4,0,2


In [11]:
# sanitize the data & engineer some features
import numpy as np

def engineer(df, fn):
  return df.apply(fn, axis=1)

def netyardsatt(att, passyards, sacks, sackyards):
  return (passyards - sackyards) / (att + sacks)

def rushingefficiency(att, yds):
  return 0 if att == 0 else yds / att

def tdtodiff(passtds, rushtds, ints, fums):
  return passtds + rushtds - ints - fums

def wrangle(df):
  df = df.copy()

  # remove "QBs" with no passes
  df = df[df['passatt'] > 0]

  # some qbs have a trailing '*' on their name
  df['player'] = df['player'].str.replace(r'\*$', '')

  # fix missing `sacks`/`sackyards` data
  df['sacks'] = df['sacks'].fillna(0)
  df['sackyards'] = df['sackyards'].fillna(0)

  # severely imbalanced classes;
  # remove everyone who doesn't have a full season of data
  vc = df['player'].value_counts()
  df = df[df['player'].isin(vc[vc >= 16].index.to_list())]

  # turn `date` into a date
  df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)

  # engineer a `season`-year feature
  df['season'] = df['date'].apply(lambda x: x.year if x.month > 6 else x.year - 1)
  
  # turn the `game` into a quintile of the season
  # (eg, games 1-4 are "early", games 17-20 are "postseason", etc)
  df['quint'] = pd.cut(df['game'],
                       bins=np.linspace(0,20,6),
                       labels=range(5)
                      ).astype(int)

  # engineer passing stats
  df['cmp%'] = engineer(df, lambda x: x['completions'] / x['passatt'])
  df['ny/a'] = engineer(df, lambda x: netyardsatt(x['passatt'],
                                                  x['passyards'],
                                                  x['sacks'],
                                                  x['sackyards']))

  # determine rushing efficiency
  df['ypc'] = engineer(df, lambda x: rushingefficiency(x['rushatt'],
                                                           x['rushyards']))
  
  # calculate Touchdown : Turnover differential
  df['td:to'] = engineer(df, lambda x: tdtodiff(x['passtds'],
                                                x['rushtds'],
                                                x['ints'],
                                                x['fumbles']))
  
  keepem = ['player', 'season', 'quint', 'home',
            'cmp%', 'passatt', 'ny/a', 'rushatt', 'ypc', 'td:to']
  df = df[keepem]
  df = df.rename(columns={'passatt': 'attempts', 'rushatt': 'carries'})

  return df

df_w = wrangle(df0)
quickpeek(df_w)

(8723, 10)
136 QBs


,player,season,quint,home,cmp%,attempts,ny/a,carries,ypc,td:to
0,Geno Smith,2013,2,1,0.400000,10,1.909091,1,2.000000,-1
1,Ryan Tannehill,2013,2,0,0.651163,43,7.454545,3,7.333333,1
2,Brandon Weeden,2013,2,1,0.600000,40,7.953488,2,2.500000,-1
3,Joe Flacco,2013,2,1,0.685714,35,6.405405,4,1.750000,0
4,Matt Flynn,2013,2,0,0.500000,20,3.777778,2,2.000000,-3


In [12]:
df_w.to_csv('clean.txt', index=False)